In [1]:
import numpy as np
import igl
import meshplot as mp

import time
from IPython.display import IFrame

In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [3]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 4})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [4]:
# retreives the index of the closest point to point in points
def find_closest_point(point, points):
    return np.argmin(np.linalg.norm(points - point, 2, axis=1))

# Setting up the Constraints
def constraints(points, normals, eps):
    """
    add constraint points to point clou
    
    """
    fip = np.empty(points.shape[0])
    fim = np.empty(points.shape[0])
    pip = np.empty(points.shape)
    pim = np.empty(points.shape)
    for i in range(points.shape[0]):
        pi = points[i]
        epsp = eps
        epsm = eps
        pipi = pi + eps * normals[i]
        while (find_closest_point(pipi, points) != i):
            epsp = epsp / 2
            pipi = pi + epsp * normals[i]
        pip[i,:] = pipi
        fip[i] = epsp
        
        pimi = pi - eps * normals[i]
        while (find_closest_point(pimi, points) != i):
            epsm = epsm / 2
            pimi = pi - epsm * normals[i]
        pim[i,:] = pimi
        fim[i] = -epsm
        
    fi = np.zeros(points.shape[0])
    # concat the original points with the added constraint points
    p = np.concatenate((points, pip, pim))
    f = np.concatenate((fi, fip, fim))
    return p,f

box_max = np.max(pi, axis=0)
box_min = np.min(pi, axis=0)
box_diag = np.linalg.norm(box_max-box_min, 2)
eps = 0.01 * box_diag

p, f = constraints(pi, ni, eps)

plt1 = mp.plot(p[np.where(f==0)], shading={"point_size": 4, "point_color": "red"})
plt1.add_points(p[np.where(f>0)], shading={"point_size": 4, "point_color": "blue"}) # outside
plt1.add_points(p[np.where(f<0)], shading={"point_size": 4, "point_color": "green"}) # inside
plt1.save("Fig1_constraint.html")
IFrame(src='Fig1_constraint.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Plot saved to file Fig1_constraint.html.


# MLS function

In [5]:
def closest_points(point, points, h, p=2):
    """
    Linear search to find all points that has distance < h to the given point
    """
    return np.where(np.linalg.norm(point - points, p, axis=1) < h)[0]
    
def MLS(x, pt, fs, h, polyDegree):
    """
    Moving least square algorithm
 
    Args:
        x (np.array(n**3)): bounding uniform tet grid 
        pt (np.ndarray((#V, 3))): constraint point cloud 
        fs (np.array(#V): corresponding implicit function values for point cloud
        h (float): Wendland radius
        polyDegree (0/1/2): polynomial degree
 
    Returns:
        np.ndarray((n**3, 3)): bounding uniform tet grid 
        T: tet faces
        np.array(n**3): interpolated value computed at each tet grid point
    """
    
    deg = {0:1, 1:4, 2:10}[polyDegree]
    # B (#V x deg): global basis matrix of constraints 1,x,y,z,x^2,y^2,z^2,xy,xz,yz
    B = np.vstack((np.ones(pt.shape[0]), 
                  pt[:,0], pt[:,1], pt[:,2], 
                  np.multiply(pt[:,0],pt[:,1]), np.multiply(pt[:,0],pt[:,2]), np.multiply(pt[:,1],pt[:,2]),
                  np.power(pt[:,0],2), np.power(pt[:,1],2), np.power(pt[:,2],2) 
                  )).T[:, 0:deg]

    # MLS on each tet grid point
    values = np.empty(x.shape[0])
    for i in range(x.shape[0]):
        xi = x[i]
        # find adj nodal points falls within the radius that have wendland weight > 0, adj_pts << n**3
        adjs = closest_points(xi, pt, h)
        # if not enough points, assign a large positive value to the grid point
        if (adjs.shape[0] < np.ceil(deg)):
            values[i] = 100000
        else:
            # B local matrix of adj pts, (adj_pts x deg), where adj_pts >= deg
            Bx = B[adjs,:]
            # constraint vector evaluated at each interpolant, (adj_pts x 1)
            dx = fs[adjs]
            # diagonal weights for each adj pts, (adj_pts x adj_pts)
            cis = pt[adjs]        
            dist = np.linalg.norm(xi-cis, 2, axis=1)
            W = (1-dist/h)**4 * (4*dist/h+1)
            W = np.diag(W)
            # Solving coeff vector (deg x 1) from the linear system: (B'WB)a = B'W dx
            # B'WB a : (deg x deg)(deg x 1)
            # B'W dx : (deg x adj_pts)(adj_pts x 1)
            ax = np.linalg.solve(Bx.T@W@Bx, Bx.T@W@dx)
            bx = np.array([1, xi[0], xi[1], xi[2], xi[0]*xi[1], xi[0]*xi[2], xi[1]*xi[2], xi[0]**2, xi[1]**2, xi[2]**2])[0:deg]
            values[i] = bx.dot(ax)
    return values

In [6]:
# filter and plot the component with largest length
def filter_largest_comp(sf):
    """
    Filter out the largest connected component

    Args:
        sf: all surfaces

    Returns:
        surfaces of the largest connected component
    """
    comp = igl.facet_components(sf)
    maxi = 0
    maxval = -np.inf
    maxind = np.array([])
    for i in range(np.max(comp)):
        val = comp[np.where(comp==i)[0]].shape[0]
        if (val > maxval):
            maxi = i
            maxval = val
    return sf[np.where(comp==maxi)[0]]

# Marching to Extract the Surface

## Experiment with degree = 0

In [7]:
# global configs here
wendlandRadius = 18
polyDegree = 0 # 0, 1, 2
resolution = (50,50,50)

# construct grid
box_max = np.max(p, axis=0)
box_min = np.min(p, axis=0)
box_diag = np.linalg.norm(box_max-box_min, 2) # approx 147
x, T = tet_grid(resolution, box_min-0.05*box_diag, box_max+0.05*box_diag)

values = MLS(x, p, f, wendlandRadius, polyDegree)

inds = np.zeros((resolution[0]*resolution[1]*resolution[2], 3))
inds[values >= 0, :] = np.array([0,1,0])
inds[values < 0, :] = np.array([1,0,0])

plt2 = mp.plot(x, c=inds, shading={"point_size": 3,"width": 800, "height": 800}) 
plt2.save("Fig2_deg0_pt.html")
IFrame(src='Fig2_deg0_pt.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120954…

Plot saved to file Fig2_deg0_pt.html.


In [8]:
# Marcing tet to extract zero-level surface
sv1, sf1, _, _ = igl.marching_tets(x, T, values, 0)
filtered_sf = filter_largest_comp(sf1)
plt3 = mp.plot(sv1, filtered_sf, shading={"wireframe": True})
plt3.save("Fig3_deg0.html")
IFrame(src='Fig3_deg0.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0816192…

Plot saved to file Fig3_deg0.html.


In [9]:
# global configs here
wendlandRadius = 18
polyDegree = 1 # 0, 1, 2
resolution = (50,50,50)

# construct grid
box_max = np.max(p, axis=0)
box_min = np.min(p, axis=0)
box_diag = np.linalg.norm(box_max-box_min, 2) # approx 147
x, T = tet_grid(resolution, box_min-0.05*box_diag, box_max+0.05*box_diag)

values = MLS(x, p, f, wendlandRadius, polyDegree)

inds = np.zeros((resolution[0]*resolution[1]*resolution[2], 3))
inds[values >= 0, :] = np.array([0,1,0])
inds[values < 0, :] = np.array([1,0,0])

plt4 = mp.plot(x, c=inds, shading={"point_size": 3,"width": 800, "height": 800})    
plt4.save("Fig4_deg1_pt.html")
IFrame(src='Fig4_deg1_pt.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120954…

Plot saved to file Fig4_deg1_pt.html.


In [10]:
# Marcing tet to extract zero-level surface
sv1, sf1, _, _ = igl.marching_tets(x, T, values, 0)
filtered_sf = filter_largest_comp(sf1)
plt5 = mp.plot(sv1, filtered_sf, shading={"wireframe": True})
plt5.save("Fig5_deg1.html")
IFrame(src='Fig5_deg1.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120954…

Plot saved to file Fig5_deg1.html.


## Experiment with degree = 2

In [11]:
# global configs here
wendlandRadius = 18
polyDegree = 2 # 0, 1, 2
resolution = (50,50,50)

# construct grid
box_max = np.max(pi, axis=0)
box_min = np.min(pi, axis=0)
box_diag = np.linalg.norm(box_max-box_min, 2) # approx 147
eps = 0.01 * box_diag
x, T = tet_grid(resolution, box_min-0.05*box_diag, box_max+0.05*box_diag)
p, f = constraints(pi, ni, eps)
values = MLS(x, p, f, wendlandRadius, polyDegree)

inds = np.zeros((resolution[0]*resolution[1]*resolution[2], 3))
inds[values >= 0, :] = np.array([0,1,0])
inds[values < 0, :] = np.array([1,0,0])
plt6 = mp.plot(x, c=inds, shading={"point_size": 3,"width": 800, "height": 800})    
plt6.save("Fig6_deg2_pt.html")
IFrame(src='Fig6_deg2_pt.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Plot saved to file Fig6_deg2_pt.html.


In [12]:
# Marcing tet to extract zero-level surface
sv1, sf1, _, _ = igl.marching_tets(x, T, values, 0)
filtered_sf = filter_largest_comp(sf1)
plt7 = mp.plot(sv1, filtered_sf, shading={"wireframe": True})
plt7.save("Fig7_deg2.html")
IFrame(src='Fig7_deg2.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Plot saved to file Fig7_deg2.html.


## Experiment Observation:

### Selected surface regions to compare the influence of PolyDegree

Global variables: radius=18, resolution=50x50x50

From left to right: PolyDegree=0,1,2
<div>
<img src="img/deg0.png" alt="drawing" width="200"/>
<img src="img/deg1.png" alt="drawing" width="200"/>
<img src="img/deg2.png" alt="drawing" width="200"/>
</div>
<div>
<img src="img/deg0_noise.png" alt="drawing" width="200"/>
<img src="img/deg1_noise.png" alt="drawing" width="200"/>
<img src="img/deg2_noise.png" alt="drawing" width="200"/>
</div>

### Higher Order Interpolation Captures Higher Curvature Surface:

I test the algorithm with different polyDegree for interpolation. Comparing the local shape of specific surface areas (like the ear tips and the face shape of the cat), it is implied that higher order interpolation MLS can reconstruct surface with higher curvature, and surface reveals more local details as degree increases. 

### Higher Order Interpolation is Prone to Noise / Missing Data: 

More local details usually means more noise. For higher degree interpolation, MLS is more prone to noise, which happens when there is not enough points. As we can see, as the original point cloud lack points at the bottom (around the cat's arms/paws), for polyDegree = 2, the reconstruction suffers greatly from the noise at the bottom and the arms/paws, resulting in undesired artifacts and unsmooth surface.


# Implementing a spatial index to accelerate neighbor calculations

In [13]:
# spatial index binning vertices into their enclosing grid cells 
def spatial_ind_at_res(pts, x, n):
    """
    Build a two-way relationship between points and each cell of the grid
    Args:
        pts (#V, 3): point cloud
        x (#N, 3): volumeric grid
        n: (#Nx, #Ny, #Nz) resolution at each axis of the grid
    Returns:
        attr_ind (#Nx, #Ny, #Nz, int[]): index of cell -> indexes of points enclosed in the cell
        spat_ind (#V, 3): point index -> index of enclosing cell, using this we don't need to pass the grid around
    """
    origin_point = x[0,:]
    DIFF = np.divide(x[-1,:] - origin_point, n) # assume the grid is ordered from lower corner to upper corner
    spat_ind = np.empty(pts.shape)
    attr_ind = np.empty(n, dtype=object)
    for i in range(pts.shape[0]):
        ind = np.floor((pts[i] - origin_point) / DIFF).astype(int)
        spat_ind[i,:] = ind
        if (attr_ind[ind[0]][ind[1]][ind[2]] == None):
            attr_ind[ind[0]][ind[1]][ind[2]] = []
        attr_ind[ind[0]][ind[1]][ind[2]].append(i)
    return attr_ind, spat_ind
    
def add_points_to_attr_ind(points, pts, x, n, attr_ind):
    """
    add a bunch of new points to a existing attr_ind
    """
    origin_point = x[0,:]
    DIFF = np.divide(x[-1,:] - origin_point, n) # assume the grid is ordered from lower corner to upper corner
    inds = np.floor((points - origin_point) / DIFF).astype(int)
    for i in range(inds.shape[0]):
        if (attr_ind[inds[i,0]][inds[i,1]][inds[i,2]] == None):
            attr_ind[inds[i,0]][inds[i,1]][inds[i,2]] = []
        attr_ind[inds[i,0]][inds[i,1]][inds[i,2]].append(pts.shape[0]+i) # append the new point to the end
    pts = np.vstack((pts, points))
    return pts, attr_ind
    
def find_closest_point_acc(point, pts, x, n, attr_ind, eps):
    """
    given the spatial index of all cloud points
    for a given arbitary point
    find the closest point within certain blocks
    Args:
        point (1, 3): query point
        pts (#V, 3): point cloud
        x (#N, 3), n: (#Nx, #Ny, #Nz): volumeric grid and its resolution
        eps: the maximum distance in which the closest point is considered
        attr_ind: precomputed attribute index of the grid
    """    
    min_d = np.inf
    min_i = -1
    # to find the closest one. if not found, return -1
    origin_point = x[0,:]
    # assume the grid is ordered from lower corner to upper corner
    DIFF = (x[-1,:] - origin_point) / n
    # restricting exploration to certain neighboring r x r x r cells
    r = np.ceil(eps/DIFF).astype(int)                        
    # find the enclosing index, e.g. [n, m, p]
    location = np.floor((point - origin_point) / DIFF).astype(int)
    # test all points in neighbouring boxes, e.g. [n+-r, m+-r. p+-r], clipped search range within the grid
    search_range = np.clip([location-r, location+r], 0, np.array(n)-1)
    for i in range(search_range[0,0],search_range[1,0]):
        for j in range(search_range[0,1],search_range[1,1]):
            for k in range(search_range[0,2],search_range[1,2]):
                eptis = attr_ind[i][j][k] # points contained in the searching voxel
                if (eptis == None):
                    continue
                for epti in eptis:
                    val = np.linalg.norm(point - pts[epti],2)
                    if val < min_d:
                        min_d = val
                        min_i = epti
    return min_i

def closest_points_acc(point, pts, x, n, attr_ind, h):
    """
    given the spatial index of all cloud points
    for a given point on the grid, compute all cloud points close enough
    Args:
        point (1, 3): query point on the tet grid
        pts (#V, 3): point cloud
        x (#N, 3), n: (#Nx, #Ny, #Nz): volumeric grid and its resolution
        attr_ind: precomputed attribute index of the grid
    """
    min_is = []
    origin_point = x[0,:]
    # assume the grid is ordered from lower corner to upper corner, (1,3)
    DIFF = (x[-1,:] - origin_point) / n
    # restricting exploration to certain neighboring r x r x r cells, whose radius should include all possible points
    r = np.ceil(h/DIFF).astype(int)                        
    location = np.floor((point - origin_point) / DIFF).astype(int)
    search_range = np.clip([location-r, location+r], 0, np.array(n)-1)
    for i in range(search_range[0,0],search_range[1,0]):
        for j in range(search_range[0,1],search_range[1,1]):
            for k in range(search_range[0,2],search_range[1,2]):
                eptis = attr_ind[i][j][k] # points contained in the searching voxel
                if (eptis == None):
                    continue
                for epti in eptis:
                    val = np.linalg.norm(point - pts[epti],2)
                    if val < h:
                        min_is.append(epti)
    return np.array(min_is)

In [14]:
# Now the grid need to be preconstructed before adding constraints
def constraints_acc(points, normals, eps, x, res, attr_ind):
    """
    add constraint points to point cloud
    """
    fip = np.empty(points.shape[0])
    fim = np.empty(points.shape[0])
    pip = np.empty(points.shape)
    pim = np.empty(points.shape)
    for i in range(points.shape[0]):
        pi = points[i];
        epsp = eps
        epsm = eps
        pipi = pi + eps * normals[i]
        while (find_closest_point_acc(pipi, points, x, res, attr_ind, 2*eps) != i):
            epsp = epsp / 2
            pipi = pi + epsp * normals[i]
        fip[i] = epsp
        pip[i,:] = pipi
        
        pimi = pi - eps * normals[i]
        while (find_closest_point_acc(pimi, points, x, res, attr_ind, 2*eps) != i):
            epsm = epsm / 2
            pimi = pi - epsm * normals[i]
        fim[i] = -epsm
        pim[i,:] = pimi
    
    # concat the original points with the added constraint points
    fi = np.zeros(points.shape[0])
    f = np.concatenate((fi, fip, fim))
    points, attr_ind = add_points_to_attr_ind(np.vstack((pip, pim)), points, x, res, attr_ind)
    return points, f, attr_ind

# accelerated MLS
def MLS_acc(x, pt, fs, h, polyDegree, attr_ind):
    """
    Moving least square algorithm accelerated
    Args:
        x (np.array(n**3)): bounding uniform tet grid 
        pt (np.ndarray((#V, 3))): constraint point cloud 
        fs (np.array(#V): corresponding implicit function values for point cloud
        h (float): Wendland radius
        polyDegree (0/1/2): polynomial degree
    Returns:
        np.ndarray((n**3, 3)): bounding uniform tet grid 
        T: tet faces
        np.array(n**3): interpolated value computed at each tet grid point
    """
    deg = {0:1, 1:4, 2:10}[polyDegree]
    # B (#V x deg): global basis matrix of constraints 1,x,y,z,x^2,y^2,z^2,xy,xz,yz
    B = np.vstack((np.ones(pt.shape[0]), 
                  pt[:,0], pt[:,1], pt[:,2], 
                  np.multiply(pt[:,0],pt[:,1]), np.multiply(pt[:,0],pt[:,2]), np.multiply(pt[:,1],pt[:,2]),
                  np.power(pt[:,0],2), np.power(pt[:,1],2), np.power(pt[:,2],2) 
                  )).T[:, 0:deg]

    # MLS on each tet grid point
    values = np.empty(x.shape[0]);
    for i in range(x.shape[0]):
        xi = x[i]
        # find adj nodal points falls within the radius that have wendland weight > 0, adj_pts << n**3
        adjs = closest_points_acc(xi, pt, x, resolution, attr_ind, h)
        if (adjs.shape[0] < np.ceil(deg)):
            values[i] = 100000;
        else:
            # B local matrix of adj pts, (adj_pts x deg), where adj_pts >= deg
            Bx = B[adjs,:]
            # constraint vector evaluated at each interpolant, (adj_pts x 1)
            dx = fs[adjs]
            # diagonal weights for each adj pts, (adj_pts x adj_pts)
            cis = pt[adjs]        
            dist = np.linalg.norm(xi-cis, 2, axis=1)
            W = (1-dist/h)**4 * (4*dist/h+1)
            W = np.diag(W)
            # Solving coeff vector (deg x 1) from the linear system: (B'WB)a = B'W dx
            # B'WB a : (deg x deg)(deg x 1)
            # B'W dx : (deg x adj_pts)(adj_pts x 1)
            ax = np.linalg.solve(Bx.T@W@Bx, Bx.T@W@dx)
            bx = np.array([1, xi[0], xi[1], xi[2], xi[0]*xi[1], xi[0]*xi[2], xi[1]*xi[2], xi[0]**2, xi[1]**2, xi[2]**2])[0:deg]
            values[i] = bx.dot(ax)
    return values

In [15]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)

## global configs here
wendlandRadius = 18
polyDegree = 2 # 0, 1, 2
resolution = (30,30,30)

## Common settings for the grid
box_max = np.max(pi, axis=0)
box_min = np.min(pi, axis=0)
box_diag = np.linalg.norm(box_max-box_min, 2)
eps = 0.01 * box_diag
x, T = tet_grid(resolution, box_min-0.05*box_diag, box_max+0.05*box_diag)

### Using attr_ind to accelerate
# Calculate the time elapsed
start = time.time()
attr_ind, _ = spatial_ind_at_res(pi, x, resolution)
p1, f1, attr_ind = constraints_acc(pi, ni, eps, x, resolution, attr_ind)
end = time.time()
print("Time elapsed for accelerated adding constraints is: ", end - start, " seconds")
values1 = MLS_acc(x, p1, f1, wendlandRadius, polyDegree, attr_ind)
end = time.time()
print("Total accelerated time elapsed is: ", end - start, " seconds")

### Not using attr_ind to accelerate
# Calculate the time elapsed
start = time.time()
p2, f2 = constraints(pi, ni, eps)
end = time.time()
print("Time elapsed for unaccelerated adding constraints is: ", end - start, " seconds")
values2 = MLS(x, p2, f2, wendlandRadius, polyDegree)
end = time.time()
print("Total Unaccelerated time elapsed is: ", end - start, " seconds")

Time elapsed for accelerated adding constraints is:  0.01709723472595215  seconds
Total accelerated time elapsed is:  8.588510990142822  seconds
Time elapsed for unaccelerated adding constraints is:  0.006242036819458008  seconds
Total Unaccelerated time elapsed is:  0.847771167755127  seconds


In [16]:
inds = np.zeros((resolution[0]*resolution[1]*resolution[2], 3))
inds[values1 >= 0, :] = np.array([1,1,0])
inds[values1 < 0, :] = np.array([1,0,0])
plt8 = mp.plot(x, c=inds, shading={"point_size": 3,"width": 400, "height": 400})  
plt8.save("Fig8_compare_interpl_acc.html")
IFrame(src='Fig8_compare_interpl_acc.html', width=400, height=400)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Plot saved to file Fig8_compare_interpl_acc.html.


In [17]:
inds2 = np.zeros((resolution[0]*resolution[1]*resolution[2], 3))
inds2[values2 >= 0, :] = np.array([1,1,0])
inds2[values2 < 0, :] = np.array([1,0,1])
plt9 = mp.plot(x, c=inds2, shading={"point_size": 3,"width": 400, "height": 400}) 
plt9.save("Fig9_compare_interpl.html")
IFrame(src='Fig9_compare_interpl.html', width=400, height=400)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Plot saved to file Fig9_compare_interpl.html.


In [18]:
# plot and compare the constraint and interpolated value
plt10 = mp.plot(p1[np.where(f1==0)], shading={"point_size": 4, "point_color": "red"})
plt10.add_points(p1[np.where(f1>0)], shading={"point_size": 4, "point_color": "blue"}) # outside
plt10.add_points(p1[np.where(f1<0)], shading={"point_size": 4, "point_color": "green"}) # inside
plt10.save("Fig10_compare_const_acc.html")
IFrame(src='Fig10_compare_const_acc.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Plot saved to file Fig10_compare_const_acc.html.


In [19]:
plt11 = mp.plot(p2[np.where(f2==0)], shading={"point_size": 4, "point_color": "red"})
plt11.add_points(p2[np.where(f2>0)], shading={"point_size": 4, "point_color": "blue"}) # outside
plt11.add_points(p2[np.where(f2<0)], shading={"point_size": 4, "point_color": "green"}) # inside
plt11.save("Fig11_compare_const.html")
IFrame(src='Fig11_compare_const.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

Plot saved to file Fig11_compare_const.html.


## Outputs

### Accelarated:

Time elapsed for accelerated adding constraints is:  0.016447782516479492  seconds

Total accelerated time elapsed is:  8.462278127670288  seconds

### Unaccelarated:

Time elapsed for unaccelerated adding constraints is:  0.005535125732421875  seconds

Total Unaccelerated time elapsed is:  0.8456680774688721  seconds

### difference in results:

## Reasoning:

Sadly, the "accelerated" version is much more slower than the unaccelerated one. However, since the outcome are the same, there should be no bugs in the implementation of the spatial index. 

The reason why spatial-index fails to work is probably because Python for loop is super slow, comparing to Numpy apis which send the task to GPU and well optimized. However, it is good to have a spatial index implemented and optimized, so that the algorithm scale well as resolution / point cloud size increases.

# Using a non-axis-aligned grid

Denote the rectangular point cloud matrix with SVD decomposition $M = U*S*V^T$ 

The idea is to find a linear operator, so that after the operation, the eigenvectors of the row space are canonical basis. 

Observe that $R = V$ does the work:

$$
M*R = (U*S*V^T)*R 
= U*S*(V^T*V)
= U*S*I_{3x3}
$$


In [20]:
pi2, v2 = igl.read_triangle_mesh("data/luigi.off")

# move the point cloud to the center of the origin
center = np.mean(pi2, axis=0)
pi2 = pi2 - center # (#V x 3)

# svd the matrix
S = pi2
_,_,Vt = np.linalg.svd(S)

# apply inv rotation to straighten the point cloud, V^-1 = V^T 
# the idea is to make eigenvector in row space to be canonical basis: M*Vt^T = (U*S*Vt)*Vt^T = U*S*I_{3x3}
pi2 = np.dot(pi2, Vt.T)

# need to recompute the per-vertex normal here
ni2 = igl.per_vertex_normals(pi2, v2)

In [21]:
# add constraints
box_max = np.max(pi2, axis=0)
box_min = np.min(pi2, axis=0)
box_diag = np.linalg.norm(box_max-box_min, 2) # approx 147
eps = 0.01 * box_diag
resolution = (40, 40, 40)
# construct the axis-aligned grid that contains the straightened point cloud
x_r, T = tet_grid(resolution, box_min - 0.03 * box_diag, box_max + 0.03 * box_diag)

## Check the alignment indeed works
plt12 = mp.plot(x_r, shading={"point_size": 2, "point_color": "green", "width": 800, "height": 800})
plt12.add_points(pi2, shading={"point_size": 4, "point_color": "red"})
plt12.save("Fig12_axis_rotation.html")
IFrame(src='Fig12_axis_rotation.html', width=800, height=800)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.6777229…

Plot saved to file Fig12_axis_rotation.html.


# Extracting the surface

More results are already shown in previous sections. Here I provide a plot for luigi surface reconstruction at 30x30x30, wendlandRadius = 12, polyDegree = 0

<img src="img/luigi_30.png" alt="drawing" width="200"/>

In [22]:
# global configs here, need some tuning here
wendlandRadius = 12
polyDegree = 0 # 0, 1, 2

## Unaccelarated version
start = time.time()
p3, f3 = constraints(pi2, -ni2, eps) # here need to flip the sign of normals, idk why, probably due to the aligning
end = time.time()
print("Non-Accelerated time elapsed for adding constraints is: ", end - start, " seconds")
fvs3 = MLS(x_r, p3, f3, wendlandRadius, polyDegree)
end = time.time()
print("Total non-accelerated time elapsed is: ", end - start, " seconds")

# plot interpolated function
inds = np.zeros((resolution[0]*resolution[1]*resolution[2], 3))
inds[fvs3 >= 0, :] = np.array([0,1,0])
inds[fvs3 < 0, :] = np.array([1,0,0])
plt13 = mp.plot(x_r, c=inds, shading={"point_size": 3,"width": 800, "height": 800})
plt13.save("Fig13_luigi_interpl.html")
IFrame(src='Fig13_luigi_interpl.html', width=800, height=800)

Non-Accelerated time elapsed for adding constraints is:  0.1659221649169922  seconds
Total non-accelerated time elapsed is:  8.232270240783691  seconds


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.6777229…

Plot saved to file Fig13_luigi_interpl.html.


In [23]:
# Marcing tet to extract zero-level surface
sv3, sf3, _, _ = igl.marching_tets(x_r, T, fvs3, 0)
filtered_sf3 = filter_largest_comp(sf3);
plt14 = mp.plot(sv3, filtered_sf3, shading={"wireframe": False})
plt14.save("Fig14_luigi_surface.html")
IFrame(src='Fig14_luigi_surface.html', width=800, height=800)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.1739711…

Plot saved to file Fig14_luigi_surface.html.


# (Optional) Interpolating and Approximating Implicit Surfaces from Polygon Soup 

the paper uses the normal to define a linear function for each point cloud point: 
**the signed distance to the tangent plane** at the point.

Then the values of these linear functions are interpolated by MLS. 
Implement Section 3.3 of the paper and 
append to your report a description of the method and 
how it compares to the original point-value-based approach. 

## Quotes

"forcing the interpolating function to behave like a prescribed function (in the neighborhood of a polygon), 
as opposed to a prescribed constant value
...

...instead of using the moving least-squares method to blend between *constant values* associated with each polygon (or point), 
we blend between *functions* associated with them.
"

## Methods

$\phi_k$ is a constraint value associated to the polygon/point (q_k). 

The linear system to solve:

$$
\mathbf{w}(\mathbf{x}, \mathbf{p}) * c = W(\mathbf{x}, \mathbf{p}) * \mathbf{s}(\mathbf{x})
$$

$W * s(\mathbf{x})$: (N x N)(N x 1) = (N x 1),

$\mathbf{w} * c$: (Nx1)(1x1) = (N x 1)


Instead of adding constriant points with non-negative constriant values, we only use the zero constraint at point clouds $p_i$, 

For each $x_i$ on the grid, find all points $p_i$ close enough; These points form a "polygon" (on 2d this is obvious). For that polygon we have a linear system: 

$$
\begin{align}
S_k(x) &= \phi_k +(x−q_k)^T \hat{n_k} \\
    &= \psi_{0k}+\psi_{x_k}x+\psi_{y_k}y+\psi_{z_k}z \\
    &= 
    \left[\begin{array}1
    1 & x_1 & y_1 & z_1 \\
    1 & x_2 & y_2 & z_2 \\
    \vdots & \vdots & \vdots & \vdots \\
    \end{array}\right]
    \psi
    \\
\end{align}
$$

where $\phi_k$ is constraint so it is just 0; $q_k$ is an arbitrary point on the polygon/point, and $\psi$ is the resulting polynomial coefficient vector.

In [24]:
def MLS_poly(x, pt, ni, h, polyDegree):
    """
    Moving least square algorithm
 
    Args:
        x (np.array(n**3)): bounding uniform tet grid 
        pt (np.ndarray((#V, 3))): constraint point cloud 
        ni (np.array(#V): corresponding nodal normal
        h (float): Wendland radius
        polyDegree (0/1/2): polynomial degree

        attr_ind, spat_ind: acceleration
    Returns:
        np.ndarray((n**3, 3)): bounding uniform tet grid 
        T: tet faces
        np.array(n**3): interpolated value computed at each tet grid point
    """
    deg = {0:1, 1:4, 2:10}[polyDegree]
    B = np.vstack((np.ones(pt.shape[0]), 
                  pt[:,0], pt[:,1], pt[:,2], 
                  np.multiply(pt[:,0],pt[:,1]), np.multiply(pt[:,0],pt[:,2]), np.multiply(pt[:,1],pt[:,2]),
                  np.power(pt[:,0],2), np.power(pt[:,1],2), np.power(pt[:,2],2) 
                  )).T[:, 0:deg]
    # MLS on each tet grid point
    values = np.empty(x.shape[0]);
    for i in range(x.shape[0]):
        xi = x[i]
        # find adj nodal points falls within the radius that have wendland weight > 0, adj_pts << n**3
        adjs = closest_points(xi, pt, h)
        # if not enough points, assign a large positive value to the grid point
        if (adjs.shape[0] < np.ceil(deg / 2)):
            values[i] = 100000;
        else:
            # Bx: local basis matrix, (adj_pts,deg)
            Bx = B[adjs,:]
            
            # pis: adj points (adj_pts,3)
            pis = pt[adjs]
            
            # nis: normal vectors of adj pts (adj_pts,3) - a bunch of row vectors
            nis = ni[adjs]
            
            # dx: constraint vec eval at each interpolant, (adj_pts,1)
            dx = np.diag((pis-xi)@(nis).T) # (adj_pts,3)(adj_pts,3)'=(adj_pts,adj_pts), we only use the dot products on diagonal (a_i^T bi_)
            
            # weight matrix W(xi, p), (adj_pts,adj_pts)
            dist = np.linalg.norm(xi-pis, 2, axis=1)
            W = (1-dist/h)**4 * (4*dist/h+1)
            W = np.diag(W)
            
            # Solving coeff vector (deg,1) from the linear system: (B'WB)a = B'W dx
            # B'WB a : (deg,deg)(deg,1); # B'W dx : (deg,adj_pts)(adj_pts,1)
            ax = np.linalg.solve(Bx.T@W@Bx, Bx.T@W@dx)

            # compute interpolant value
            bx = np.array([1, xi[0], xi[1], xi[2], xi[0]*xi[1], xi[0]*xi[2], xi[1]*xi[2], xi[0]**2, xi[1]**2, xi[2]**2])[0:deg]
            values[i] = bx.dot(ax)
    return values

box_max = np.max(pi2, axis=0)
box_min = np.min(pi2, axis=0)
box_diag = np.linalg.norm(box_max-box_min, 2) # approx 147
eps = 0.01 * box_diag

# Plot of the grid with nodes colored according to their implicit function values.
wendlandRadius = 0.05 * box_diag
polyDegree = 0 # 0, 1, 2

fvs = MLS_poly(x_r, pi2, ni2, wendlandRadius, polyDegree)

inds = np.zeros((resolution[0]*resolution[1]*resolution[2], 3))
inds[fvs >= 0, :] = np.array([0,1,0])
inds[fvs < 0, :] = np.array([1,0,0])
plt15 = mp.plot(x_r, c=inds, shading={"point_size": 3,"width": 800, "height": 800})
plt15.save("Fig15_luigi_poly.html")
IFrame(src='Fig15_luigi_poly.html', width=800, height=800)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.6777229…

Plot saved to file Fig15_luigi_poly.html.


## Estimate a normal for results obtained with single dataset.

Per-surface normal is computed and attached to the barycenter of each triangle.

<img src="img/luigi_poly_normal_lines.png" alt="drawing" width="200"/>

In [25]:
# Marcing tet to extract zero-level surface
sv2, sf2, _, _ = igl.marching_tets(x_r, T, fvs, 0)
plt16 = mp.plot(sv2, sf2, shading={"wireframe": False})
sn2 = igl.per_face_normals(sv2, sf2, np.ones(3)*1e-4)
sf2_b = igl.barycenter(sv2, sf2)
plt16.add_lines(sf2_b, sf2_b+sn2)

plt16.save("Fig16_luigi_poly_surface.html")
IFrame(src='Fig16_luigi_poly_surface.html', width=800, height=800)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.3415794…

Plot saved to file Fig16_luigi_poly_surface.html.


# (Optional) Screened Poisson Surface Reconstruction 

(inward pointing) normal field of the boundary of a solid can be interpreted as the gradient of the solid’s indicator function

Find indicator function $\chi$ whose gradient best approximates 

$$
\min_{\chi}\int \|\nabla \chi - \mathbf{v}(\mathbf x)\|^2dx \\
$$

Solving the linear system:

$$
\Delta \chi = \nabla \cdot \mathbf{v}
$$

The discrete Laplacian for 3D:

$$
\left[
\begin{array}
1 9 & -1 & 0 & \cdots & -1 & 0 & \cdots & -1 & 0 & \cdots \\
 -1 & -9 & -1 & \cdots & 0 & -1 & \cdots & 0 & -1 & \cdots \\
\end{array}
\right]
$$

## Results from the meshlab

**Note: need to zoom-in / zoom-out to visualize the result from meshlab, for some reason meshplot cannot render the full mesh from meshlab except from certain view angles**

The left figure is generated by meshlab; the right figure is generated by vanilla MLS with polyDegree=2. 

To make resonable comparison, we choose the resolution so that the MLS result and Screened Poisson Surface Reconstruction result are close (~4000-5000).

### 1.The Screened Poisson Reconstruction performs much better on the boundary

With boundary condition (Dirichlet) properly set, the boundary of SPR result is smooth. In comparison since the cat model does not have points on the bottom, the MLS algorithm lack information so does not produce meaningful result.

### 2.The SPR provide a smoother result

Even with smaller number of surfaces, the SPR algorithm generates smoother "normal changes" on the surface, resulting in a much smoother surface.

### 3. The SPR does not suffer from Runge's phenomenon 

On the surface of MLS construct, there are artifacts and bumps. Also, as we can observe from previous implicit functions, noisy points outside the surface exists, this is because we are using uniform interpolation and Runge's phenomenon. 

In comparison, SPR does not suffer from this problem.


### Poisson Reconstruction Result:

<img src="img/Poisson_result.png" alt="drawing" width="200"/>


In [26]:
# import pymeshlab as ms
# ms.generate_surface_reconstruction_screened_poisson()

pi_c, v_c = igl.read_triangle_mesh("data/cat_meshlab.off")
plt17 = mp.plot(pi_c, v_c)
plt17.save("Fig17_poisson_compare.html")
IFrame(src='Fig17_poisson_compare.html', width=1200, height=800)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(49.714805…

Plot saved to file Fig17_poisson_compare.html.
